In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df_true=pd.read_csv('/content/drive/MyDrive/katia/True.csv')
df_fake=pd.read_csv('/content/drive/MyDrive/katia/Fake.csv')
df_true["label"] = "real"
df_fake["label"] = "fake"

In [7]:
frames = [df_true,df_fake]
df = pd.concat(frames)
df

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,12/31/2017,real
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,12/29/2017,real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,12/31/2017,real
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,12/30/2017,real
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,12/29/2017,real
...,...,...,...,...,...
23445,COORDINATED? Yet Another Hack Reporter Claims ...,A coordinated effort to spew anti-Trump propag...,politics,12/31/2017,fake
23446,BARBRA STREISAND Gives Up On Dream Of Impeachi...,Barbra Streisand was an Obama sycophant and on...,politics,12/31/2017,fake
23447,WATCH: SENATOR LINDSEY GRAHAM DROPS BOMBSHELL…...,Everyone suspected the sketchy Steele Dossier ...,politics,12/31/2017,fake
23448,BARBRA STREISAND Gives Up On Dream Of Impeachi...,Barbra Streisand was an Obama sycophant and on...,left-news,12/31/2017,fake


In [8]:
from sklearn.utils import shuffle
df = shuffle(df)
df['label']=df['label'].apply(lambda x: 1 if x=='fake' else 0)
labels=df.label
labels

18543    1
6636     1
6762     1
10264    0
1842     1
        ..
19723    0
16568    1
3428     0
1690     1
15831    0
Name: label, Length: 44867, dtype: int64

In [ ]:
def Count_capsword(text):
  cw = 0
  cw = sum(map(str.isupper, str(text).split()))
  return cw

#df['CapsWord_title']=Count_capsword(df['title'])
#df['CapsWord_text']=Count_capsword(df['text'])


Cleaning data

In [9]:
column_n = ['title', 'text','subject','date','label']
remove_c = ['subject']
categorical_features = []
target_col = ['label']
text_f = ['title', 'text']

# Clean Datasets
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)


# drop row with shirt text
def null_process(feature_df):
    for col in text_f:
        if len(feature_df[col])<7:
          feature_df = feature_df.drop(feature_df.index(col))
    return feature_df

def clean_dataset(df):
    #impute null values
    df = null_process(df)
    return df
    
# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    return text


def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return  text


#https://www.thepythoncode.com/article/fake-news-classification-in-python

In [55]:
df1 = df.copy()

In [56]:
df1['date']= df1['date'].astype('datetime64[ns]')
df1

,title,text,subject,date,label
18543,BILL BENNETT RIPS Into Never Trumpers…The Voic...,Bill Bennett has been the voice of reason for ...,politics,2016-08-20,1
6636,Trump’s Taking Credit For Inaugural Singer’s ...,"With the inauguration fast approaching, Donald...",News,2017-01-04,1
6762,LOL! CNN Host Don Lemon Tells Viewers: “We wil...,The most laughable news network in America j...,left-news,2017-04-04,1
10264,Mansion where Obama to stay in Havana 'built t...,HAVANA (Reuters) - The mansion in Havana where...,politicsNews,2016-03-18,0
1842,LA Father Kills Son For Being Gay (VIDEO),Less than two weeks before his father used a s...,News,2016-04-03,1
...,...,...,...,...,...
19723,Trump to speak in 'tough terms' about North Ko...,NEW YORK (Reuters) - President Donald Trump wi...,worldnews,2017-09-19,0
16568,"In No Particular Order, Sarah Palin Uses Rand...",As much as you may want Sarah Palin to drift o...,News,2016-06-27,1
3428,U.S. House to vote on Republican Dodd-Frank re...,(Reuters) - The U.S. House of Representatives ...,politicsNews,2017-06-02,0
1690,BREAKING FL VOTER FRAUD ALERT: “Tens Of Thousa...,https://twitter.com/TEN_GOP/status/79389017105...,left-news,2016-11-02,1


In [57]:
clean_dataset(df1)
# apply preprocessing on text through apply method by calling the function nltk_preprocess
df1["text"] = df1.text.apply(clean_text)
df1["text"] = df1.text.apply(nltk_preprocess)
# apply preprocessing on title through apply method by calling the function nltk_preprocess
df1["title"] = df1.title.apply(clean_text)
df1["title"] = df1.title.apply(nltk_preprocess)
df1

,title,text,subject,date,label
18543,bill bennett rip never trumpersthe voice reaso...,bill bennett voice reason conservative incredi...,politics,2016-08-20,1
6636,trump taking credit inaugural singer album sal...,inauguration fast approaching donald trump try...,News,2017-01-04,1
6762,lol cnn host lemon tell viewer insult intellig...,laughable news network america proved unreliab...,left-news,2017-04-04,1
10264,mansion obama stay havana built impress,havana reuters mansion havana u president bara...,politicsNews,2016-03-18,0
1842,la father kill son gay video,le two week father used shotgun end life amir ...,News,2016-04-03,1
...,...,...,...,...,...
19723,trump speak tough term north korea first un sp...,new york reuters president donald trump speak ...,worldnews,2017-09-19,0
16568,particular order sarah palin us random word pr...,much may want sarah palin drift obscurity know...,News,2016-06-27,1
3428,u house vote republican doddfrank reform bill ...,reuters u house representative vote broad bill...,politicsNews,2017-06-02,0
1690,breaking fl voter fraud alert ten thousand mai...,httpstwittercomten_gopstatus793890171058085888...,left-news,2016-11-02,1


In [59]:
df1['text'].sort_values()

17235                                                     
8970                                                      
4446     1 7 antitrump rally raging across america make...
4435     1 7 antitrump rally raging across america make...
14091    1 christian2 gay3 fear lifestyle put odds bibl...
                               ...                        
14741    zurich reuters switzerland stop seizing income...
18720    zurich reuters switzerland stripped libyan mus...
18027    zurich reuters switzerland sunday called quick...
11392    zurich reuters three people died separate aval...
14397    zurich reuters united state consular office sw...
Name: text, Length: 44867, dtype: object

In [61]:
df1.to_csv('/content/drive/MyDrive/katia/clean_data.csv',sep='\t')